## Setup

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AY0e-g4GXaWdYUM2NszFpyEr0kU_hY9R4qMoe0ESQ_E0veQIFkrQ769DGXo



Successfully saved authorization token.


In [2]:
import numpy as np
import geetools
from geetools import ui, cloud_mask
import os, datetime
import config as cf
import pandas as pd

cloud_mask_landsatSR = cloud_mask.landsatSR()
cloud_mask_sentinel2 = cloud_mask.sentinel2()

/Users/robmarty/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [3]:
SURVEY_NAME = 'DHS'

## Functions

In [7]:
def survey_to_fc_buffer(survey_df, buffer_size_urban, buffer_size_rural):
    '''
    Convert pandas dataframe of survey locations to a feature collection. 
    
    Inputs:
        survey_df: pandas dataframe of survey locations. Function assumes 
                   the dataframe contains (1) latitude, (2) longitude and
                   (3) uid variables. Assumes coordinates in WGS84.
    Returns:
        (feature collection)
    '''
    
    survey_fc_list = []
    
    n_rows = survey_df.shape[0]
    for i in range(0, n_rows):
        survey_df_i = survey_df.iloc[[i]]
        
        ur = survey_df_i['urban_rural'].iloc[0]
        if ur == 'U':
            buffer_size = buffer_size_urban
        elif ur == 'R':
            buffer_size = buffer_size_rural

        f_i = ee.Feature(ee.Geometry.Point([survey_df_i['longitude'].iloc[0], 
                                            survey_df_i['latitude'].iloc[0]]), 
                         {'uid': survey_df_i['uid'].iloc[0]})
        
        f_i = f_i.buffer(buffer_size)

        survey_fc_list.append(f_i)
        
    survey_fc = ee.FeatureCollection(survey_fc_list)
    
    return survey_fc

def extract_sat(survey_df, buffer_size_urban, buffer_size_rural, satellite, year):
    '''
    Extract satellite imagery to locations 
    
    Inputs:
        survey_df: pandas dataframe of survey locations. Function assumes 
                   the dataframe contains (1) latitude, (2) longitude and
                   (3) uid variables. Assumes coordinates in WGS84.
    Returns:
        (feature collection)
    '''
    
    # Prep l7 ---------------------------------------------------
    if satellite == 'l7':
        
        # Scale
        SCALE = 100 # ok to upscale
        
        # Year
        year_use = year
        
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('LANDSAT/LC07/C01/T1_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_landsatSR)\
            .median()\
            .multiply(0.0001)

        ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI');
        image = image.addBands(ndvi)
        
        BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
        
    # Prep l8 ---------------------------------------------------
    if satellite == 'l8':
        
        SCALE = 100 # ok to upscale
        
        # Year
        # landsat 8 starts in April 2013; if year is less than
        # 2014, use 2014 as year (to ensure have year before and after)
        if year < 2014:
            year_use = 2014
        else:
            year_use = year
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_landsatSR)\
            .median()\
            .multiply(0.0001)

        ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');
        image = image.addBands(ndvi)
        
        BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'NDVI']
        
    # Prep s2 ---------------------------------------------------
    if satellite == 's2':
        
        SCALE = 100 # ok to upscale
        
        # Year
        # sentinel starts in March 2017; juse use 2018
        year_use = 2018
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_sentinel2)\
            .median()\
            .multiply(0.0001)

        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI');
        image = image.addBands(ndvi)
        
        BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'NDVI']
    
    # Prep viirs ---------------------------------------------------
    if satellite == 'viirs':
        
        SCALE = 500 
        
        # Year
        # VIIRS starts in April 2012; if year is less than
        # 2013, use 2013 as year (to ensure have year before and after)
        if year < 2013:
            year_use = 2013
        else:
            year_use = year
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')\
            .filterDate(year_minus_str, year_plus_str)\
            .median()
        
        BANDS = ['avg_rad']
        
    # Prep DMSP ---------------------------------------------------
    if satellite == 'dmsp':
        
        SCALE = 1000 
        
        # Year
        # DMSP-OLS starts in 2013; if year is more than
        # 2012, use 2012 as year (to ensure have year before and after)
        if year > 2012:
            year_use = 2012
        else:
            year_use = year
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS')\
            .filterDate(year_minus_str, year_plus_str)\
            .median()
        
        BANDS = ['stable_lights', 'avg_lights_x_pct']
    
    # Prep Survey ---------------------------------------------------
    survey_fc = survey_to_fc_buffer(survey_df, buffer_size_urban, buffer_size_rural)
    
    # Extract Values ---------------------------------------------------
    vals = image.reduceRegions(collection = survey_fc,
                               reducer = ee.Reducer.mean(),
                               scale = SCALE,
                               tileScale = 8)
    
    survey_df = survey_df[['uid']]
    
    for band_i in BANDS:
        #print(band_i)
        survey_df[satellite + '_' + band_i] = vals.aggregate_array(band_i).getInfo()
        
    return survey_df

def extract_satellite_in_chunks(survey_df, buffer_size_urban, buffer_size_rural, satellite, year):
    
    vals_df_list = []
    
    for chunk_i in list(np.unique(survey_df.chunk_id)):
        print(chunk_i)

        survey_df_i = survey_df[survey_df['chunk_id'] == chunk_i]
        vals_i_df = extract_sat(survey_df_i, buffer_size_urban, buffer_size_rural, satellite, year)

        vals_df_list.append(vals_i_df)

    vals_df = pd.concat(vals_df_list)
    
    return vals_df

def extract_satellite_by_year(survey_df, buffer_size_urban, buffer_size_rural, satellite):
    
    vals_df_list = []
    
    for year_i in list(np.unique(survey_df.year)):
        print(year_i)

        survey_df_i = survey_df[survey_df['year'] == year_i]
        vals_i_df = extract_satellite_in_chunks(survey_df_i, buffer_size_urban, buffer_size_rural, satellite, year_i)

        vals_df_list.append(vals_i_df)

    vals_df = pd.concat(vals_df_list)
    
    return vals_df

def chunk_ids(total_length, chunk_size):
    n_numbers = np.ceil(total_length / chunk_size)
    n_numbers = int(n_numbers)
    
    chunk_ids = list(range(0,n_numbers)) * chunk_size
    chunk_ids.sort()
    chunk_ids = chunk_ids[:total_length]
    
    return chunk_ids

## Load/Prep Survey Data

In [8]:
survey_df = pd.read_csv(os.path.join(cf.SECURE_DATA_DIRECTORY, 'Data', SURVEY_NAME, 'FinalData - PII', 'GPS_uid_crosswalk.csv'))
survey_df = survey_df[survey_df.most_recent_survey == True]
#survey_df = survey_df.head(1000)
CHUNK_SIZE = 1000

survey_years = list(survey_df.year.unique())

In [9]:
survey_df['chunk_id'] = chunk_ids(survey_df.shape[0], CHUNK_SIZE)

## Extract Landsat 8

In [10]:
val_l8_df = extract_satellite_by_year(survey_df, 2000, 5000, 'l8')

val_l8_df.to_csv(os.path.join(cf.DROPBOX_DIRECTORY, 
                            'Data', 
                            SURVEY_NAME, 
                            'FinalData', 
                            'Individual Datasets',
                           'survey_l8.csv'),
              index = False)

val_l8_df.head()

2003
29


/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


30
2012
30
31
2014
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


ValueError: Length of values does not match length of index

## Extract Landsat 7

In [ ]:
val_l7_df = extract_satellite_by_year(survey_df, 2000, 5000, 'l7')

val_l7_df.to_csv(os.path.join(cf.DROPBOX_DIRECTORY, 
                            'Data', 
                            SURVEY_NAME, 
                            'FinalData', 
                            'Individual Datasets',
                           'survey_l7.csv'),
              index = False)

val_l7_df.head()

## Extract Sentinel

In [ ]:
val_s2_df = extract_satellite_by_year(survey_df, 2000, 5000, 's2')

val_s2_df.to_csv(os.path.join(cf.DROPBOX_DIRECTORY, 
                            'Data', 
                            SURVEY_NAME, 
                            'FinalData', 
                            'Individual Datasets',
                           'survey_s2.csv'),
              index = False)

val_s2_df.head()

## Extract VIIRS

In [ ]:
val_viirs_df = extract_satellite_by_year(survey_df, 2000, 5000, 'viirs')

val_viirs_df.to_csv(os.path.join(cf.DROPBOX_DIRECTORY, 
                            'Data', 
                            SURVEY_NAME, 
                            'FinalData', 
                            'Individual Datasets',
                           'survey_viirs.csv'),
              index = False)

val_viirs_df.head()

## Extract DMSP-OLS

In [ ]:
val_dmsp_df = extract_satellite_by_year(survey_df, 2000, 5000, 'dmsp')

val_dmsp_df.to_csv(os.path.join(cf.DROPBOX_DIRECTORY, 
                            'Data', 
                            SURVEY_NAME, 
                            'FinalData', 
                            'Individual Datasets',
                           'survey_dmsp.csv'),
              index = False)

val_dmsp_df.head()

In [ ]:
#import folium
#from folium import plugins

#mapid = survey_i_fc.getMapId()

#map = folium.Map(location=[80.773137, 7.873592])
#folium.TileLayer(
#    tiles=mapid['tile_fetcher'].url_format,
#    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#    overlay=True,
#    name='border',
#  ).add_to(map)

#map.add_child(folium.LayerControl())


In [ ]:
survey_df.head()